In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def calculate_ema(self, data, length=9, source='Close', offset=0):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=9).mean()  # Smoothing length of 9 as specified
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate EMA
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None
    
    def calculate_ema_21(self, data):
        """
        Calculate EMA with a period of 21 days.

        Args:
            data (pd.DataFrame): Historical price data
        
        Returns:
            pd.Series: EMA 21 values
        """
        return self.calculate_ema(data, length=21, source='Close', offset=0)
    
    def calculate_ema_50(self, data):
        """
        Calculate EMA with a period of 50 days.

        Args:
            data (pd.DataFrame): Historical price data
        
        Returns:
            pd.Series: EMA 50 values
        """
        return self.calculate_ema(data, length=50, source='Close', offset=0)

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean(),
                                # Adding EMA 21 and EMA 50 to stock_info
                                'EMA_21': self.calculate_ema_21(hist_data).iloc[-1],
                                'EMA_50': self.calculate_ema_50(hist_data).iloc[-1]
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")


if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/HAPPSTMNDS.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=HAPPSTMNDS.NS&crumb=CdOdjA2vtBF
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GVT&D.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GVT%26D.NS&crumb=CdOdjA2vtBF



NSE analysis complete. Results saved to stock_analysis\daily_results\nse_analysis_20250109.xlsx

NSE Stocks with RSI >= 40:
ATLASCYCLE (Atlas Cycles (Haryana) Limited): RSI = 100.00, Price = ₹102.56
BGRENERGY (BGR Energy Systems Limited): RSI = 100.00, Price = ₹116.67
BLUECOAST (Blue Coast Hotels Limited): RSI = 100.00, Price = ₹21.84
VISASTEEL (VISA Steel Limited): RSI = 100.00, Price = ₹45.48
GVT&D (N/A): RSI = 100.00, Price = ₹1990.20
AARVEEDEN (Aarvee Denims and Exports Ltd.): RSI = 100.00, Price = ₹129.00
FSC (Future Supply Chain Solutions Limited): RSI = 96.77, Price = ₹2.49
INDOTHAI (Indo Thai Securities Limited): RSI = 94.57, Price = ₹1648.95
PTCIL (PTC Industries Limited): RSI = 92.88, Price = ₹17392.15
SVLL (Shree Vasu Logistics Limited): RSI = 91.62, Price = ₹386.95
AXISCADES (AXISCADES Technologies Limited): RSI = 90.15, Price = ₹809.15
SRF (SRF Limited): RSI = 84.05, Price = ₹2673.90
AVONMORE (Avonmore Capital & Management Services Limited): RSI = 83.05, Price = ₹27.24
GO

$512399.BO: possibly delisted; no price data found  (1d 2024-11-10 19:22:52.854807 -> 2025-01-09 19:22:52.854807)


Error fetching data for 512441: Expecting value: line 1 column 1 (char 0)
Error fetching data for 512453: Expecting value: line 1 column 1 (char 0)


...

Error fetching data for 530759: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530825: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530789: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530809: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530845: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530859: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530821: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530829: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530879: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530883: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530889.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530889.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530953.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530953.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530897.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530897.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530951.BO?mo

Error fetching data for 532354: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532395: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532413: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532485: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532493: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532553: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532629: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532645: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532674: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532720: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532744: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532779: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532829: Expecting value: line 1 column 1 (char 0)
Error fetching data for 532853: Expect

429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/539787.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=539787.BO&crumb=CdOdjA2vtBF
$539730.BO: possibly delisted; no price data found  (1d 2024-11-10 19:29:14.146026 -> 2025-01-09 19:29:14.146026)
$539725.BO: possibly delisted; no price data found  (1d 2024-11-10 19:29:14.023158 -> 2025-01-09 19:29:14.023158)
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/539785.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=539785.BO&crumb=CdOdjA2vtBF
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/539784.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomai

Error fetching data for 541178: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541144: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541206: Expecting value: line 1 column 1 (char 0)Error fetching data for 541228: Expecting value: line 1 column 1 (char 0)

Error fetching data for 541337: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541353: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541299: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541401: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541347: Expecting value: line 1 column 1 (char 0)
Error fetching data for 541445: Expecting value: line 1 column 1 (char 0)

BSE analysis complete. Results saved to stock_analysis\daily_results\bse_analysis_20250109.xlsx

BSE Stocks with RSI >= 40:
531663 (Yuvraaj Hygiene Products Limited): RSI = 100.00, Price = ₹5.59
539266 (Concrete Infra & Media Limited): RSI = 100.00, Price = ₹

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def calculate_ema(self, data, length=9, source='Close', offset=0):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=9).mean()  # Smoothing length of 9 as specified
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate EMA
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")


if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EMAMIPAP.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EMAMIPAP.NS&crumb=CdOdjA2vtBF
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/KRONOX.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=KRONOX.NS&crumb=CdOdjA2vtBF
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/KRN.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=KRN.NS&crumb=CdOdjA2vtBF
$KREBSBIO.NS: possibly delisted; no price data found  (1d 2024-11-10 19:35:25.059262 -> 2025-01-09 19:35:25.059262)
$KRITINUT.NS: possibly delisted; no price d


NSE analysis complete. Results saved to stock_analysis\daily_results\nse_analysis_20250109.xlsx

NSE Stocks with RSI >= 40:
ATLASCYCLE (Atlas Cycles (Haryana) Limited): RSI = 100.00, Price = ₹102.56
VISASTEEL (VISA Steel Limited): RSI = 100.00, Price = ₹45.48
BLUECOAST (Blue Coast Hotels Limited): RSI = 100.00, Price = ₹21.84
GVT&D (GE Vernova T&D India Ltd): RSI = 100.00, Price = ₹1990.20
AARVEEDEN (Aarvee Denims and Exports Ltd.): RSI = 100.00, Price = ₹129.00
BGRENERGY (BGR Energy Systems Limited): RSI = 100.00, Price = ₹116.67
FSC (Future Supply Chain Solutions Limited): RSI = 96.77, Price = ₹2.49
INDOTHAI (Indo Thai Securities Limited): RSI = 94.57, Price = ₹1648.95
PTCIL (PTC Industries Limited): RSI = 92.88, Price = ₹17392.15
SVLL (Shree Vasu Logistics Limited): RSI = 91.62, Price = ₹386.95
AXISCADES (AXISCADES Technologies Limited): RSI = 90.15, Price = ₹809.15
SRF (SRF Limited): RSI = 84.05, Price = ₹2673.90
AVONMORE (Avonmore Capital & Management Services Limited): RSI = 83.

$523716.BO: possibly delisted; no price data found  (1d 2024-11-10 19:38:45.312698 -> 2025-01-09 19:38:45.312698)
$523708.BO: possibly delisted; no price data found  (1d 2024-11-10 19:38:45.303563 -> 2025-01-09 19:38:45.303563)
$523704.BO: possibly delisted; no price data found  (1d 2024-11-10 19:38:45.215389 -> 2025-01-09 19:38:45.215389)
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/523796.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=523796.BO&crumb=Edge%3A+Too+Many+Requests
$523694.BO: possibly delisted; no price data found  (1d 2024-11-10 19:38:45.395770 -> 2025-01-09 19:38:45.395770)
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/523836.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&sym

Error fetching data for 523732: Expecting value: line 1 column 1 (char 0)


...

Error fetching data for 530007: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530011: Expecting value: line 1 column 1 (char 0)
Error fetching data for 527001: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530075.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530075.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530073.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530073.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530077.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530077.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530093.BO?mo

Error fetching data for 530019: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530043: Expecting value: line 1 column 1 (char 0)
Error fetching data for 530017: Expecting value: line 1 column 1 (char 0)


429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530117.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530117.BO&crumb=Edge%3A+Too+Many+Requests
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530119.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530119.BO&crumb=CdOdjA2vtBF
$530077.BO: possibly delisted; no price data found  (1d 2024-11-10 19:40:46.706259 -> 2025-01-09 19:40:46.706259)
429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/530125.BO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=530125.BO&crumb=CdOdjA2vtBF
$530111.BO: possibly delisted


BSE analysis complete. Results saved to stock_analysis\daily_results\bse_analysis_20250109.xlsx

BSE Stocks with RSI >= 40:
532368 (Brightcom Group Limited): RSI = 100.00, Price = ₹10.28
539266 (Concrete Infra & Media Limited): RSI = 100.00, Price = ₹3.99
533499 (Bharatiya Global Infomedia Limited): RSI = 100.00, Price = ₹4.74
534535 (Bio Green Papers Limited): RSI = 100.00, Price = ₹41.63
512405 (N/A): RSI = 100.00, Price = ₹15.13
531663 (Yuvraaj Hygiene Products Limited): RSI = 100.00, Price = ₹5.59
539032 (Fraser and Company Limited): RSI = 100.00, Price = ₹9.12
531769 (PFL Infotech Limited): RSI = 100.00, Price = ₹5.61
533163 (ARSS Infrastructure Projects Limited): RSI = 100.00, Price = ₹28.68
511401 (N/A): RSI = 100.00, Price = ₹20.27
539544 (Abhishek Infraventures Limited): RSI = 100.00, Price = ₹7.83
532090 (Vandana Knitwear Limited): RSI = 100.00, Price = ₹11.22
541701 (N/A): RSI = 100.00, Price = ₹343.00
505029 (Atlas Cycles (Haryana) Limited): RSI = 100.00, Price = ₹97.65
53

In [6]:
import yfinance as yf

symbol = 'AARTECH.NS'
period = '1mo'

# Fetch the historical data
ticker = yf.Ticker(symbol)
hist_data = ticker.history(period=period)

# Display the historical data
print(hist_data)

                                 Open        High        Low       Close  \
Date                                                                       
2024-12-10 00:00:00+05:30   82.400002   83.000000  78.019997   79.120003   
2024-12-11 00:00:00+05:30   80.699997   80.699997  76.940002   77.129997   
2024-12-12 00:00:00+05:30   78.389999   78.949997  77.000000   77.699997   
2024-12-13 00:00:00+05:30   77.989998   80.099998  75.500000   79.559998   
2024-12-16 00:00:00+05:30   80.900002   81.080002  77.070000   77.500000   
2024-12-17 00:00:00+05:30   77.919998   78.169998  76.000000   77.699997   
2024-12-18 00:00:00+05:30   77.669998   77.669998  74.199997   75.169998   
2024-12-19 00:00:00+05:30   74.239998   78.919998  73.110001   78.900002   
2024-12-20 00:00:00+05:30   79.010002   82.839996  79.010002   82.519997   
2024-12-23 00:00:00+05:30   85.779999   86.379997  79.500000   80.339996   
2024-12-24 00:00:00+05:30   80.000000   82.000000  80.000000   80.250000   
2024-12-26 0

In [10]:
from datetime import datetime 

start_date = datetime(2023, 12, 10)
end_date = datetime(2024, 1, 10)

hist_data = ticker.history(
    start=start_date,
    end=end_date,
    interval="1d"
)

print(hist_data)

                                Open       High        Low      Close  Volume  \
Date                                                                            
2023-12-11 00:00:00+05:30  49.000000  49.000000  47.400002  48.166664   15462   
2023-12-12 00:00:00+05:30  48.833332  48.833332  47.333332  47.549999   13290   
2023-12-13 00:00:00+05:30  48.333332  49.400002  47.333332  48.549999   20982   
2023-12-14 00:00:00+05:30  48.333332  48.666664  47.666664  48.200001   12666   
2023-12-15 00:00:00+05:30  48.166664  48.333332  46.666664  47.433334   18066   
2023-12-18 00:00:00+05:30  48.966667  49.599998  46.000000  47.200001  274809   
2023-12-19 00:00:00+05:30  47.366665  47.950001  44.849998  45.666664   52569   
2023-12-20 00:00:00+05:30  45.333332  46.983334  43.500000  43.900002  112848   
2023-12-21 00:00:00+05:30  44.016666  44.299999  42.416664  43.400002   97257   
2023-12-22 00:00:00+05:30  43.616665  44.650002  43.616665  44.466667  275493   
2023-12-26 00:00:00+05:30  4